# Parameter Estimation for a Scalar Nonlinear Differential Equation

This tutorial demonstrates how to estimate the parameters of a scalar nonlinear differential equation using Python.

## Overview
We will:
1. Define a nonlinear ODE model.
2. Generate synthetic data using `solve_ivp` and add noise.
3. Estimate parameters by fitting the data using `least_squares`.
4. Plot the results to compare the noisy data and the estimated model.

## Implementation

In [3]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.integrate import solve_ivp
from scipy.optimize import least_squares

# Define the logistic growth ODE model
def nonlinear_ode(t, y, a, b):
    return a * y * (1 - b * y)

# Generate synthetic data
def generate_data(a_true, b_true, y0, t_span, t_eval, noise_std=0.05):
    sol = solve_ivp(nonlinear_ode, t_span, [y0], t_eval=t_eval, args=(a_true, b_true))
    assert sol.success, "Integration failed"
    y_noisy = sol.y[0] + np.random.normal(0, noise_std, size=sol.y.shape[1])
    return t_eval, y_noisy

# Function to fit the parameters
def estimate_parameters(t, y_meas, y0):
    def residuals(params):
        a, b = params
        sol = solve_ivp(nonlinear_ode, (t[0], t[-1]), [y0], t_eval=t, args=(a, b))
        assert sol.success, "Integration failed"
        return sol.y[0] - y_meas
    
    result = least_squares(residuals, x0=[0.5, 0.1])
    return result.x


In [4]:
# True parameters
a_true = 0.5
b_true = -0.3
y0 = 1.0
t_span = (0, 5)
t_eval = np.linspace(t_span[0], t_span[1], 50)

# Generate data
t, y_meas = generate_data(a_true, b_true, y0, t_span, t_eval)

# Plot generated data
plt.figure(figsize=(8, 5))
plt.scatter(t, y_meas, label="Noisy Data", color='red')
plt.xlabel("Time")
plt.ylabel("y")
plt.legend()
plt.title("Generated Data for Nonlinear ODE")
plt.show()

AssertionError: Integration failed

In [ ]:


# Estimate parameters
a_est, b_est = estimate_parameters(t, y_meas, y0)

# Plot results
plt.figure(figsize=(8, 5))
plt.scatter(t, y_meas, label="Noisy Data", color='red')
sol_est = solve_ivp(nonlinear_ode, t_span, [y0], t_eval=t, args=(a_est, b_est))
plt.plot(t, sol_est.y[0], label=f"Fitted Model (a={a_est:.2f}, b={b_est:.2f})", linestyle='dashed')
plt.xlabel("Time")
plt.ylabel("y")
plt.legend()
plt.title("Parameter Estimation for Nonlinear ODE")
plt.show()

print(f"Estimated Parameters: a = {a_est:.2f}, b = {b_est:.2f}")


```python

```

## Explanation
- We define the nonlinear ODE: \( \frac{dy}{dt} = a y^2 + b y \).
- We generate synthetic data by solving the ODE numerically and adding noise.
- We estimate the parameters by solving an inverse problem using `curve_fit`.
- Finally, we visualize the results to compare the noisy data with the fitted model.

## Results
The estimated parameters should be close to the true values \( a = 0.5 \), \( b = -0.3 \), but some deviation is expected due to noise.
